In [1]:
pip install "requests_html"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 10.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
# pip install "requests.html"

# Import Libraries
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd
import functools
import time

# store the url in a variable
scraping_control_variables = {
    "city_name": "Hsinchu",
    "geo_code": "13806499",
    "data_offset_lower_limit": 0,
    "data_offset_upper_limit": 6330,
    "starting_page_num": 0,
    "page_size": 30,
}

In [40]:
# Function to get URL for every page

def get_url(gc, do, city):
    data_offset_var = "-oa" + str(do)
    if do == 0:
        data_offset_var = ""
    url = f"https://www.tripadvisor.com/RestaurantSearch-g{gc}{data_offset_var}-a_date.2023__2D__03__2D__05-a_people.2-a_time.20%3A00%3A00-a_zur.2023__5F__03__5F__05-{city}.html"
    print("URL to be scraped: ", "\n", url, "\n")
    return url

In [29]:
# Function to get soup content

def get_soup_content(gc, city, do):
    url = get_url(gc, do, city)
    # start the HTML session
    print("HTML session started")
    r = HTMLSession()
    response_obj = r.get(url, verify=False)
    soup_content = BeautifulSoup(response_obj.content, "html.parser")
    return soup_content

In [41]:
def get_card(rest_cnt, soup_content):
    card_tag = f"{rest_cnt}_list_item"
    print(f"Scraping item number: {card_tag}")
    card = soup_content.find("div", {"data-test": card_tag})
    return card

In [42]:
# scrape & parse restaurants pagewise till data_offset_upper_limit is reached
def parse_tripadvisor(scv):
    restaurants_scraped = []
    current_data_offset, current_page_number = scv["data_offset_lower_limit"],scv["starting_page_num"]
    while current_data_offset <= scv["data_offset_upper_limit"]:
        print(f"Scraping started for Page Number: {current_page_number} with Dataoffset: {current_data_offset}\n")
        soup_content = get_soup_content(scv["geo_code"], scv["city_name"], current_data_offset)
        page_restaurant_data = get_parsed_restaurant_data_for_page(current_page_number, scv["page_size"], soup_content, current_data_offset)
        restaurants_scraped += page_restaurant_data
        print(f"Scraping completed for Page Number: {current_page_number} with Dataoffset: {current_data_offset}\n")
        current_page_number = current_page_number + 1
        current_data_offset = current_data_offset + 30
    return restaurants_scraped

In [43]:
# Iterate over page's soup_content to parse all cards and give a list parsed_restaurant_data_for_page having card_restaurant_data
def get_parsed_restaurant_data_for_page(page_num, page_size, soup_content, current_data_offset):
    page_start_offset = (page_num * page_size) + 1
    page_end_offset = page_start_offset + page_size
    parsed_restaurant_data_for_page = []
    for rest_cnt in range(page_start_offset, page_end_offset):
        card = get_card(rest_cnt, soup_content)
        if card is None:
            break
        card_restaurant_data = get_restaurant_data_from_card(card, rest_cnt, current_data_offset, page_num)
        parsed_restaurant_data_for_page.append(card_restaurant_data)
    return parsed_restaurant_data_for_page

In [71]:
def get_restaurant_data_from_card(card, rest_cnt, current_data_offset, page_num):
    restaurant_data = {
        "title": scrape_title(card),
        "cuisines": scrape_cuisines(card),
        "reviews": scrape_reviews(card),
        "star_rating": scrape_star_ratings(card),
        "price_range": scrape_price(card),
        "page_number": page_num,
        "data_offset": current_data_offset,
        "restaurant_serial_number": rest_cnt,
    }

    return restaurant_data

In [70]:
def scrape_star_ratings(card):
    star_rating = card.find_all("svg", class_="UctUV d H0")
    scraped_star_ratings = (star_rating[0]["aria-label"] if len(star_rating) >= 1 else None)
    
    # Check if the returned value contains "$" and return empty string if it does
    if scraped_star_ratings is None:
        return None
    return scraped_star_ratings

def scrape_price(card):
    cu_1 = card.find("div", class_="hBcUX XFrjQ mIBqD")
    try:
        cu = cu_1.find("span", class_="SUszq").get_text()
    except AttributeError:
        cu = None
    if cu is None:
        return None
    return cu if "$" in cu else None

    
def scrape_reviews(card):
    reviews = card.find_all("span", class_="IiChw")
    scraped_reviews = reviews[-1].text if len(reviews) >= 1 else None
    return scraped_reviews


def scrape_cuisines(card):
    cu_1 = card.find("div", class_="hBcUX XFrjQ mIBqD")
    try:
        cu = cu_1.find("span", class_="SUszq").get_text()
    except AttributeError:
        cu = None
    if cu is None:
        return None
    return "" if "$" in cu else cu


def scrape_title(card):
    title = card.find_all("div", class_="RfBGI")
    scraped_title = None if len(title) < 1 else title[0].text
    return scraped_title

In [72]:
def save_to_csv(restaurants_scraped):
    # finally, store the output into a csv file
    print("storing the data in csv")
    output_df = pd.DataFrame(restaurants_scraped)
    print(output_df.head(50))
#     output_df.drop_duplicates(inplace=True)
    output_df.to_csv("ta_hsinchu_restaurants_scraped.csv", index= False)
    print("csv stored")

In [73]:
def scrape_and_save(scraping_control_variables):
    restaurants_scraped = parse_tripadvisor(scraping_control_variables)
    save_to_csv(restaurants_scraped)
    return restaurants_scraped

In [74]:
# disable secure certificate warnings
requests.packages.urllib3.disable_warnings()
restaurants_scraped = scrape_and_save(scraping_control_variables)

# Print Dataframe
scraped_df = pd.DataFrame(restaurants_scraped)
print("Restaurant Data Scraped:\n",scraped_df.head(20) )

Scraping started for Page Number: 0 with Dataoffset: 0

URL to be scraped:  
 https://www.tripadvisor.com/RestaurantSearch-g13806499-a_date.2023__2D__03__2D__05-a_people.2-a_time.20%3A00%3A00-a_zur.2023__5F__03__5F__05-Hsinchu.html 

HTML session started
Scraping item number: 1_list_item
5.0 of 5 bubbles
Scraping item number: 2_list_item
4.5 of 5 bubbles
Scraping item number: 3_list_item
4.5 of 5 bubbles
Scraping item number: 4_list_item
4.0 of 5 bubbles
Scraping item number: 5_list_item
4.0 of 5 bubbles
Scraping item number: 6_list_item
4.5 of 5 bubbles
Scraping item number: 7_list_item
4.5 of 5 bubbles
Scraping item number: 8_list_item
4.0 of 5 bubbles
Scraping item number: 9_list_item
4.5 of 5 bubbles
Scraping item number: 10_list_item
4.5 of 5 bubbles
Scraping item number: 11_list_item
4.0 of 5 bubbles
Scraping item number: 12_list_item
4.0 of 5 bubbles
Scraping item number: 13_list_item
4.5 of 5 bubbles
Scraping item number: 14_list_item
4.5 of 5 bubbles
Scraping item number: 15_l

In [75]:
print("Total Restaurants scraped:\t", len(scraped_df))

print("Restaurants scraped:\t", scraped_df.head(20))

Total Restaurants scraped:	 540
Restaurants scraped:	                                                 title          cuisines  \
0          1. Chilliesine Indian Restaurant - Hsinchu            Indian   
1                                     2. Din Tai Fung  Taiwanese, Soups   
2                3. Wang Steak (Hsinchu Beida Branch)                     
3                                    4. The Promenade             Asian   
4                                          5. Le Cafe              Cafe   
5                          6. Top One Pot - Guangming    Chinese, Asian   
6                  7. ikki (Hsinchu Guangming Branch)          Japanese   
7                                 8. A Cut Steakhouse        Steakhouse   
8    9. Chamonix teppanyaki (Zhubei Guangming Branch)                     
9                              10. Eatogether Hsinchu                     
10           11. Si Chuan Duan Chun Zhen Beef Noodles    Chinese, Asian   
11                         12. Fleur Lis-Recip

In [62]:
scraped_df.to_csv('restaurants.csv')